<a href="https://colab.research.google.com/github/prateekrao/Breast-Cancer-Detection/blob/main/DenseNet_90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install --upgrade --force-reinstall --no-deps kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list
! kaggle datasets download -d rprateek/breasthistopathologyimages
! unzip -d dataset breasthistopathologyimages.zip

In [4]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, Flatten, GlobalAveragePooling2D
from keras.applications.densenet import DenseNet169, DenseNet121
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l1, l2, l1_l2
import matplotlib.pyplot as plt

In [8]:
gen = ImageDataGenerator(
    rescale = 1/255,
    # samplewise_center = True,
    # samplewise_std_normalization = True
)

In [9]:
train_set = gen.flow_from_directory(
    "/content/dataset/split-data/train/",
    target_size = (128,128),
    class_mode = 'sparse',
    shuffle = True,
    seed = 865,
    color_mode = 'rgb'
)


Found 194266 images belonging to 2 classes.


In [10]:
test_set = gen.flow_from_directory(
    "/content/dataset/split-data/test/",
    target_size = (128,128),
    class_mode = 'sparse',
    shuffle = False,
    color_mode = 'rgb'
)

Found 41629 images belonging to 2 classes.


In [20]:
valid_set = gen.flow_from_directory(
    "/content/dataset/split-data/valid/",
    target_size = (128,128),
    class_mode = 'sparse',
    shuffle = False,
    color_mode = 'rgb'
)

Found 41629 images belonging to 2 classes.


In [21]:
def build_model():

  densenet = DenseNet121(include_top=False, weights = 'imagenet', input_shape = (128,128,3))
  densenet.trainable = True
  model = tf.keras.models.Sequential()

  model.add(densenet)
  model.add(GlobalAveragePooling2D())
  
  model.add(Dense(512, activation='relu', kernel_regularizer=l1_l2(l1=1e-04, l2=1e-04)))
  model.add(Dropout(0.5))
  model.add(Dense(512, activation= 'relu', kernel_regularizer=l1_l2(l1 = 1e-05, l2= 1e-05)))
  model.add(Dropout(0.5))
  model.add(Dense(512, activation='relu', kernel_regularizer=l1_l2(l1=1e-03, l2=1e-03)))
  model.add(Dropout(0.5))
  model.add(Dense(512, activation='relu', kernel_regularizer=l1_l2(l1 = 1e-02, l2=1e-02)))
  model.add(Dense(2, activation='softmax'))
  
  return model

In [22]:
model = build_model()
model.summary()

29089792/29084464 [==============================] - 3s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 4, 4, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_______________________________________________________

In [23]:
opt = Adam(lr = 1e-04)

model.compile(optimizer=opt, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [24]:
checkpoint = ModelCheckpoint("/content/new-gpu-dense169-best.h5", monitor='val_loss', save_best_only= True, mode='min')
checkpoint

In [ ]:
history = model.fit(train_set, validation_data=valid_set, epochs = 10, callbacks = checkpoint, shuffle = True, batch_size = 128)

Epoch 1/10
6071/6071 [==============================] - 1963s 323ms/step - loss: 5.4087 - accuracy: 0.8872 - val_loss: 0.3968 - val_accuracy: 0.8913
Epoch 2/10
6071/6071 [==============================] - 1957s 322ms/step - loss: 0.3517 - accuracy: 0.9037 - val_loss: 0.3270 - val_accuracy: 0.9004
Epoch 3/10
6071/6071 [==============================] - 1956s 322ms/step - loss: 0.3104 - accuracy: 0.9130 - val_loss: 0.3389 - val_accuracy: 0.8956
Epoch 4/10
6071/6071 [==============================] - 1953s 322ms/step - loss: 0.2842 - accuracy: 0.9212 - val_loss: 0.3016 - val_accuracy: 0.9072
Epoch 5/10
6071/6071 [==============================] - 1955s 322ms/step - loss: 0.2589 - accuracy: 0.9302 - val_loss: 0.3445 - val_accuracy: 0.8995
Epoch 6/10
 145/6071 [..............................] - ETA: 29:53 - loss: 0.2145 - accuracy: 0.9468

In [ ]:
model.save("/content/drive/MyDrive/final_model_90-densenet.h5")

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Accuracy Graph")

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot("Loss Graph")

In [ ]:
model.evaluate(test_set)